In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [ ]:
info = get_info(kind='biosemi32')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

# Get sample data

In [ ]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=1, target_snr=200)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

In [ ]:
from scipy.sparse.csgraph import laplacian
from scipy.spatial.distance import cdist
adjacency = mne.spatial_src_adjacency(fwd['src'], verbose=0).toarray()
gradient = np.gradient(adjacency)
gradient = np.stack([abs(gradient[0]), abs(gradient[1])], axis=0).mean(axis=0)
laplace_operator = laplacian(adjacency)
B = np.diag(np.linalg.norm(leadfield, axis=0))
leadfield_cov = leadfield.T @ leadfield 
gradient_2 = (abs(np.gradient(leadfield_cov)[0]) + abs(np.gradient(leadfield_cov)[1])) / 2
gradient_2 = gradient_2 * adjacency


dist = cdist(pos, pos)      
gradient_2 = (abs(np.gradient(dist)[0]) + abs(np.gradient(dist)[1])) / 2

plt.figure()
plt.imshow(laplace_operator)
plt.colorbar()

# from scipy.ndimage import gaussian_gradient_magnitude, morphological_gradient
# gradient = gaussian_gradient_magnitude(adjacency,1)

plt.figure()
plt.imshow(gradient_2)
plt.colorbar()

# Regularization Optimizations

In [ ]:
# from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
# from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
# from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
# from invert.solvers.smap import SolverSMAP
# from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
from invert import Solver
solver = Solver(solver="S-Map")


solver.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))



In [ ]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)